In [52]:
from sklearn.metrics import accuracy_score
from scipy.special import softmax
import  alpaca.calibrator as calibrator
import numpy as np
import pandas as pd
from alpaca.dataloader.builder import build_dataset

In [2]:
import sklearn
import math
from torch.nn import functional as f
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

In [3]:
def compute_errors(n_bins, probs, labels, len_dataset, threshold):
    ece = calibrator.compute_ece(n_bins, probs, labels, len_dataset)
    sce = calibrator.compute_sce(n_bins, probs, labels)
    ace = calibrator.compute_ace(n_bins, probs, labels)
    tace = calibrator.compute_tace(threshold, probs, labels, n_bins)
    errors = {
        'ece' : ece,
        'sce' : sce,
        'ace' : ace,
        'tace' : tace
    }
    for error in errors.items():
        print(str(error[0]), ' = ', error[1])

In [4]:
mnist = build_dataset('mnist', val_size=10_000)

In [5]:
X_train, y_train = mnist.dataset('train')
X_val, y_val = mnist.dataset('val')
X_cal = X_train[48000:][:]
X_train = X_train[0:48000][:]
y_cal = y_train[48000:][:]
y_train = y_train[0:48000][:]

x_shape = (-1, 1, 28, 28)

train_ds = TensorDataset(torch.FloatTensor(X_train.reshape(x_shape)), torch.LongTensor(y_train))
val_ds = TensorDataset(torch.FloatTensor(X_val.reshape(x_shape)), torch.LongTensor(y_val))
train_loader = DataLoader(train_ds, batch_size=512)
val_loader = DataLoader(val_ds, batch_size=512)
cal_ds = TensorDataset(torch.FloatTensor(X_cal.reshape(x_shape)), torch.LongTensor(y_cal))
cal_loader = DataLoader(cal_ds, batch_size=512)
X_val.shape

(10000, 784)

In [6]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4 * 7 * 7, 10)
        )
  
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [7]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [8]:
for epoch in range(7):
    for x_batch, y_batch in train_loader: # Train for one epoch
        print('.', end='')
        prediction = model(x_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
    print('\nTrain loss on last batch', loss.item())

# Check accuracy
x_batch, y_batch = next(iter(val_loader))


class_preds = f.softmax(model(x_batch), dim=-1).detach().numpy()
predictions = np.argmax(class_preds, axis=-1)
print('Accuracy', accuracy_score(predictions, y_batch))

..............................................................................................
Train loss on last batch 0.6062002182006836
..............................................................................................
Train loss on last batch 0.27029111981391907
..............................................................................................
Train loss on last batch 0.17512232065200806
..............................................................................................
Train loss on last batch 0.137984961271286
..............................................................................................
Train loss on last batch 0.11777593940496445
..............................................................................................
Train loss on last batch 0.10504749417304993
..............................................................................................
Train loss on last batch 0.09596506506204605
Accuracy 0.974609375


In [9]:
logits_list = []
labels_list = []
for x_batch, y_batch in cal_loader:
    logits_list.append(model(x_batch))
    labels_list.append(y_batch)
logits = torch.cat(logits_list)
labels = torch.cat(labels_list)
logits.detach_()


tensor([[ 8.9512, -4.2500,  0.6798,  ..., -2.8549, -0.2391, -2.0600],
        [ 0.3614, -6.6645, -3.0572,  ..., -5.0718, -0.3500, -2.2592],
        [-1.1534, -2.4635, -4.5541,  ..., -3.0422,  4.0586,  0.1374],
        ...,
        [-1.9714,  6.2411, -1.3644,  ..., -0.1338, -0.8241, -1.2896],
        [-4.3371, -0.0501, -3.0126,  ..., -5.5163, -2.3426,  0.7490],
        [-1.5622, -3.2278,  1.4504,  ..., -2.4797, -1.2854, -0.3554]])

In [53]:
calibr = calibrator.ModelWithTempScaling(model)

In [54]:
calibr.scaling(logits, labels)

ModelWithTempScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [55]:
val_logits_list = []
val_labels_list = []
for x_batch, y_batch in val_loader:
    val_logits_list.append(model(x_batch))
    val_labels_list.append(y_batch)
val_logits = torch.cat(val_logits_list)
val_labels = torch.cat(val_labels_list)
val_logits.detach_()

tensor([[-5.9499e+00, -2.4130e+00,  3.5374e+00,  ..., -2.5871e+00,
         -1.1915e-02, -1.4973e+00],
        [-1.7524e+00,  6.0652e+00, -6.6139e-01,  ..., -1.5802e+00,
         -9.0216e-01, -1.8846e+00],
        [-3.2816e+00, -4.0238e+00, -4.9135e+00,  ...,  1.6583e+00,
         -1.2195e+00,  7.1447e+00],
        ...,
        [-6.3742e+00, -5.2211e+00, -5.5002e+00,  ...,  2.3404e+00,
          6.5512e-01,  5.1969e+00],
        [-2.6708e+00,  7.7116e+00,  5.5328e-01,  ...,  1.7785e-01,
          1.3387e-01, -3.4737e+00],
        [-1.3014e+00,  7.1492e+00,  1.6999e-01,  ..., -1.9588e+00,
          1.4840e-03, -3.1460e+00]])

In [56]:
probs = f.softmax(val_logits, dim=-1)


In [57]:
compute_errors(n_bins=15, probs=probs.numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0197])
sce  =  tensor([0.0024])
ace  =  tensor(0.0221)
tace  =  tensor(0.0119)


In [58]:
print(calibr.temperature)

Parameter containing:
tensor([0.6368], requires_grad=True)


In [59]:
temp_scaling_probs_list = []
for x_batch, y_batch in val_loader:
    temp_scaling_probs_list.append(calibr.forward(x_batch))
temp_scaling_probs = torch.cat(temp_scaling_probs_list)
compute_errors(n_bins=15, probs=temp_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0076])
sce  =  tensor([0.0014])
ace  =  tensor(0.0151)
tace  =  tensor(0.0056)


In [60]:
calibr = calibrator.ModelWithVectScaling(model, n_classes=10).float()

In [61]:
calibr.scaling(logits, labels, lr=0.001, max_iter=300)

ModelWithVectScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [62]:
vect_scaling_probs_list = []
for x_batch, y_batch in val_loader:
    vect_scaling_probs_list.append(calibr.forward(x_batch))
vect_scaling_probs = torch.cat(vect_scaling_probs_list)

In [63]:
compute_errors(n_bins=15, probs=vect_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0077])
sce  =  tensor([0.0014])
ace  =  tensor(0.0152)
tace  =  tensor(0.0074)


In [64]:
calibr.W_and_b

Parameter containing:
tensor([ 1.0551,  1.0509,  1.1472,  1.3279,  1.2181,  1.1756,  1.3048,  1.2188,
         1.4252,  1.2829, -0.0386, -0.0513, -0.0233,  0.0380, -0.0419, -0.0182,
         0.0301, -0.0224,  0.1063,  0.0213], requires_grad=True)

In [65]:
calibr = calibrator.ModelWithMatrScaling(model, n_classes=10).float()
calibr.scaling(logits, labels, lr=0.0001, max_iter=1000)

ModelWithMatrScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [66]:
matr_scaling_probs_list = []
for x_batch, y_batch in val_loader:
    matr_scaling_probs_list.append(calibr.forward(x_batch))
matr_scaling_probs = torch.cat(matr_scaling_probs_list)

In [67]:
compute_errors(n_bins=15, probs=matr_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0056])
sce  =  tensor([0.0014])
ace  =  tensor(0.0149)
tace  =  tensor(0.0066)


In [68]:
hist_binning_probs = calibrator.multiclass_histogram_binning(15, logits.numpy(), labels.numpy(), val_logits)

In [69]:
compute_errors(n_bins=15, probs=hist_binning_probs, labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0077])
sce  =  tensor([0.0015])
ace  =  tensor(0.0145, dtype=torch.float64)
tace  =  tensor(0.0105, dtype=torch.float64)
